In [1]:
from pathlib import Path
import pandas as pd

from expense_models import load_expenses, expenses_to_dataframe

In [2]:
expenses_path = Path("2025-11-07.json")
expenses = load_expenses(expenses_path)

len(expenses)

df = expenses_to_dataframe(expenses)
df


,name,account_id,yearly_value_eur,monthly_value_eur,yearly_value,monthly_value,currency,value,repeat_every,repeat_every_unit
0,Rent,activo_bank,8460.00,705.00,8460.00,705.00,EUR,705.00,1,months
1,Revolution food,activo_bank,2139.28,178.27,2139.28,178.27,EUR,41.00,1,weeks
2,Cleaning,activo_bank,1087.03,90.59,1087.03,90.59,EUR,62.50,3,weeks
3,T3,activo_bank,1038.48,86.54,1038.48,86.54,EUR,86.54,1,months
4,Contabilizei,nubank_cnpj,717.78,59.82,4428.00,369.00,BRL,369.00,1,months
5,Energy,activo_bank,426.84,35.57,426.84,35.57,EUR,35.57,1,months
6,Vodafone,activo_bank,342.00,28.50,342.00,28.50,EUR,28.50,1,months
7,Internet,activo_bank,309.00,25.75,309.00,25.75,EUR,25.75,1,months
8,Heroku - Teya,nubank_cnpj,288.00,24.00,288.00,24.00,EUR,24.00,1,months
9,ChatGPT,activo_bank,255.14,21.26,295.20,24.60,USD,24.60,1,months
